In [1]:
import networkx as nx
twitter: nx.DiGraph = nx.read_edgelist("./ego-Twitter/twitter_combined.txt.gz", create_using=nx.DiGraph)
twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

(81306, 1768149, True)

In [2]:
#サンプリングしたグラフからコミュニティ検出（louvain法）
import networkx as nx

twitter: nx.DiGraph = nx.read_edgelist("sampled_twitter_graph.txt", create_using=nx.DiGraph)

twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

twitter_comms = nx.community.louvain_communities(twitter)

print(f"検出されたコミュニティ数: {len(twitter_comms)}")

検出されたコミュニティ数: 51


In [ ]:
import os
import json # JSONライブラリを使用
import numpy as np
import shutil
# (networkx as nx もインポートされている前提)

# --- 1. 保存用フォルダの作成 ---
output_folder = "adj_json_communities_txt" # 保存先のフォルダ名
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"\n保存用フォルダ '{output_folder}' を作成しました。")
else:
    print(f"\n保存用フォルダ '{output_folder}' は既に存在します。")


# --- 2. 各コミュニティを指定の形式で .txt ファイルに保存 ---
print("各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...")

# twitter_comms が定義されている前提でループを開始
for i, community_nodes in enumerate(twitter_comms):
    if len(community_nodes) < 2:
        continue
    
    # Step 2-1: サブグラフを生成
    subgraph = twitter.subgraph(community_nodes)
    
    # Step 2-2: サブグラフから隣接行列を生成
    node_list = list(subgraph.nodes())
    adj_matrix_np = nx.to_numpy_array(subgraph, nodelist=node_list, dtype=int)
    
    # Step 2-3: NumPy配列をご希望の辞書形式に変換
    adjacency_dict = {}
    for row_idx, source_node in enumerate(node_list):
        row_dict = {}
        for col_idx, target_node in enumerate(node_list):
            row_dict[target_node] = int(adj_matrix_np[row_idx, col_idx])
        
        adjacency_dict[source_node] = row_dict
        
    # Step 2-4: ファイルパスを定義 (★拡張子を .txt に)
    file_path = os.path.join(output_folder, f"community_{i}.txt")
    
    # Step 2-5: TXTファイルとして書き出す (★json.dumpを使用)
    with open(file_path, 'w', encoding='utf-8') as f:
        # json.dumpで書き出すと、ご提示の例の形式と一致します
        # indent=2 を指定すると、例のように整形されて見やすくなります
        json.dump(adjacency_dict, f, indent=2) 

print(f"\n処理完了: {len(twitter_comms)}個のコミュニティを '{output_folder}' フォルダにTXTファイルとして保存しました。")


保存用フォルダ 'adj_json_communities_txt' を作成しました。
各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...

処理完了: 51個のコミュニティを 'adj_json_communities_txt' フォルダにTXTファイルとして保存しました。


In [2]:
import os
import json
import shutil # ファイルコピーのためにインポート

# --- 設定 ---

# 1. 検証元のフォルダ名
folder_name = "adj_json_communities_txt" 

# 2. 有向グラフと判定されたファイルの保存先フォルダ名
directed_folder_name = "directed_communities_txt" 

# --- 1. フォルダの準備 ---

# 検証元フォルダの存在確認
if not os.path.exists(folder_name):
    print(f"エラー: フォルダ '{folder_name}' が見つかりません。")
    print("先にファイル保存コードを実行してください。")
    exit()

# 保存先フォルダの準備 (存在しない場合は作成)
try:
    os.makedirs(directed_folder_name, exist_ok=True)
    print(f"有向グラフの保存先: '{directed_folder_name}' フォルダを準備しました。")
except Exception as e:
    print(f"エラー: 保存先フォルダ '{directed_folder_name}' の作成に失敗しました。 {e}")
    exit()


print(f"\nフォルダ '{folder_name}' 内のファイルを確認します...\n")

try:
    txt_files = [f for f in os.listdir(folder_name) if f.endswith('.txt')]
    if not txt_files:
        print("エラー: フォルダ内にTXTファイルが見つかりません。")
        exit()
except Exception as e:
    print(f"エラー: フォルダの読み込み中に問題が発生しました。 {e}")
    exit()

# --- 2. メイン処理: 検証とコピー ---

# 追跡用フラグ
all_are_directed = True
directed_files_count = 0

# 各TXTファイルをループして検証
for filename in txt_files:
    file_path = os.path.join(folder_name, filename)
    
    print(f"--- 検証中: {filename} ---")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            adj_dict = json.load(f)

        if len(adj_dict) < 2:
            print("  => ⚠️ スキップ (ノード数が2未満のため比較不可)。")
            print("-" * 35)
            continue

        nodes = list(adj_dict.keys())
        is_symmetric = True 
        asymmetry_proof = None 

        # 行列が対称か非対称かチェック
        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)): 
                node_a = nodes[i]
                node_b = nodes[j]
                
                if adj_dict[node_a][node_b] != adj_dict[node_b][node_a]:
                    is_symmetric = False
                    asymmetry_proof = (node_a, node_b) 
                    break 
            
            if not is_symmetric:
                break 

        # --- 検証結果の出力とファイルコピー ---
        if not is_symmetric:
            # 非対称 = 有向グラフ
            print(f"  ✅ 結果: これは有向グラフです。")
            print(f"     理由: 隣接行列が非対称(Asymmetric)です。")
            proof_a, proof_b = asymmetry_proof
            print(f"     証拠: ノード {proof_a} -> {proof_b} は [{adj_dict[proof_a][proof_b]}] ですが、")
            print(f"           ノード {proof_b} -> {proof_a} は [{adj_dict[proof_b][node_a]}] です。")
            
            # ★★★ 変更点: 有向グラフのファイルをコピー ★★★
            try:
                destination_path = os.path.join(directed_folder_name, filename)
                shutil.copy2(file_path, destination_path) # copy2はメタデータも保持します
                print(f"  => 💾 '{directed_folder_name}' フォルダにコピーしました。")
                directed_files_count += 1
            except Exception as e:
                print(f"  => ❌ エラー: ファイル '{filename}' のコピーに失敗しました。 {e}")
            # ★★★ ここまで ★★★

        else:
            # 対称 = 無向グラフ
            print(f"  ❌ 結果: これは有向グラフではありません (実質的に無向グラフです)。")
            print(f"     理由: 隣接行列が対称(Symmetric)です。")
            print(f"     詳細: すべてのノードペア(A, B)において、 A -> B への接続値と")
            print(f"           B -> A への接続値が完全に一致しています (全て相互関係です)。")
            all_are_directed = False

    except Exception as e:
        print(f"  => ❌ エラー: ファイル処理中に問題が発生しました。")
        print(f"     エラー内容: {e}")
        all_are_directed = False
    
    print("-" * 35)

# --- 3. 最終結論 ---
print("\n--- 総合結果 ---")
if all_are_directed:
    print("✅ 分析したすべてのコミュニティが、有向グラフの性質（非対称な関係）を保持していました。")
else:
    print("⚠️ 少なくとも1つのコミュニティが、完全に対称的（無向グラフ）であることが判明しました。")

# ★★★ 変更点: 保存結果のサマリー ★★★
print(f"\n--- 保存結果 ---")
print(f"有向グラフと判定された {directed_files_count} 個のファイルを '{directed_folder_name}' に保存しました。")

有向グラフの保存先: 'directed_communities_txt' フォルダを準備しました。

フォルダ 'adj_json_communities_txt' 内のファイルを確認します...

--- 検証中: community_1.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 14780587 -> 24815332 は [0] ですが、
           ノード 24815332 -> 14780587 は [1] です。
  => 💾 'directed_communities_txt' フォルダにコピーしました。
-----------------------------------
--- 検証中: community_0.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 2710231 -> 14060856 は [0] ですが、
           ノード 14060856 -> 2710231 は [1] です。
  => 💾 'directed_communities_txt' フォルダにコピーしました。
-----------------------------------
--- 検証中: community_2.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 23016199 -> 86572842 は [0] ですが、
           ノード 86572842 -> 23016199 は [1] です。
  => 💾 'directed_communities_txt' フォルダにコピーしました。
-----------------------------------
--- 検証中: community_3.txt ---
  ✅ 結果: これは有向グラフです。
     理由: 隣接行列が非対称(Asymmetric)です。
     証拠: ノード 236657680 -> 29877307 は [1] ですが、
  

In [1]:
import networkx as nx
import json
import os
from glob import glob

def convert_json_adj_dict_to_graph(file_path):
    """
    単一のJSON隣接辞書ファイルからNetworkXの有向グラフを生成する。
    重み1のエッジのみを追加する。
    """
    try:
        # 1. ファイルを読み込み、JSONデータをPythonの辞書にパース
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 2. 有向グラフオブジェクトの初期化
        G = nx.DiGraph()

        # 3. 辞書を反復処理してエッジとノードを追加
        for source_node, targets in data.items():
            # すべてのノードを確実に追加
            G.add_node(source_node)
            
            for target_node, weight in targets.items():
                G.add_node(target_node) # ターゲットノードも追加
                
                # 重みが1の場合のみエッジとして追加
                if weight == 1:
                    # ノード名は文字列として扱われるため、明示的な型変換は不要
                    G.add_edge(source_node, target_node, weight=weight)
                    
        return G

    except FileNotFoundError:
        print(f"エラー: ファイルが見つかりません - {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"エラー: ファイルのJSON形式が不正です - {file_path}")
        return None
    except Exception as e:
        print(f"予期せぬエラーが発生しました ({file_path}): {e}")
        return None

def process_multiple_files(folder_path="directed_communities_txt"):
    """
    指定されたフォルダ内のすべての.txtファイルをNetworkXグラフに変換するメイン関数。
    """
    # フォルダ内のすべての.txtファイルのパスを取得
    # globモジュールを使って、指定されたパターンに一致するすべてのファイル名を取得
    file_paths = glob(os.path.join(folder_path, '*.txt'))
    
    if not file_paths:
        print(f"警告: フォルダ '{folder_path}' 内に .txt ファイルが見つかりませんでした。")
        return []
    
    print(f"処理対象ファイル数: {len(file_paths)} 件")
    
    # グラフオブジェクトとそのファイル名を保存するリスト
    graphs_list = []
    
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        print(f"\nProcessing: {file_name}")
        
        # 変換関数の呼び出し
        graph = convert_json_adj_dict_to_graph(file_path)
        
        if graph:
            graphs_list.append({
                "filename": file_name,
                "graph": graph,
                "nodes": graph.number_of_nodes(),
                "edges": graph.number_of_edges()
            })
            print(f"  ✅ 成功: ノード数={graph.number_of_nodes()}, エッジ数={graph.number_of_edges()}")
        else:
            print(f"  ❌ 失敗: このファイルの処理はスキップされました。")
            
    return graphs_list

# --- 実行部分 ---

# 処理するフォルダのパスを指定
FOLDER_PATH = "directed_communities_txt"

# すべてのファイルを処理してグラフのリストを取得
converted_graphs = process_multiple_files(FOLDER_PATH)

# 結果の確認
if converted_graphs:
    print("\n--- 全ての処理結果 ---")
    for item in converted_graphs:
        print(f"ファイル名: {item['filename']}, ノード: {item['nodes']}, エッジ: {item['edges']}")
    
    # 最初のグラフオブジェクトにアクセスする例
    first_graph = converted_graphs[0]['graph']
    print(f"\n最初のグラフのノード (一部): {list(first_graph.nodes)[:5]}")
else:
    print("\nグラフは一つも生成されませんでした。ファイルパスと内容を確認してください。")

# ---

処理対象ファイル数: 47 件

Processing: community_1.txt
  ✅ 成功: ノード数=238, エッジ数=4232

Processing: community_0.txt
  ✅ 成功: ノード数=179, エッジ数=2797

Processing: community_2.txt
  ✅ 成功: ノード数=97, エッジ数=2009

Processing: community_3.txt
  ✅ 成功: ノード数=450, エッジ数=17247

Processing: community_7.txt
  ✅ 成功: ノード数=175, エッジ数=7526

Processing: community_6.txt
  ✅ 成功: ノード数=1587, エッジ数=62992

Processing: community_4.txt
  ✅ 成功: ノード数=308, エッジ数=2982

Processing: community_5.txt
  ✅ 成功: ノード数=74, エッジ数=908

Processing: community_13.txt
  ✅ 成功: ノード数=1701, エッジ数=62123

Processing: community_12.txt
  ✅ 成功: ノード数=255, エッジ数=6240

Processing: community_10.txt
  ✅ 成功: ノード数=99, エッジ数=1441

Processing: community_38.txt
  ✅ 成功: ノード数=82, エッジ数=391

Processing: community_39.txt
  ✅ 成功: ノード数=165, エッジ数=2932

Processing: community_11.txt
  ✅ 成功: ノード数=128, エッジ数=1921

Processing: community_29.txt
  ✅ 成功: ノード数=107, エッジ数=2272

Processing: community_15.txt
  ✅ 成功: ノード数=673, エッジ数=20474

Processing: community_14.txt
  ✅ 成功: ノード数=326, エッジ数=8765

Proce

In [1]:
import os
import json # JSONファイルを読み込むため
import networkx as nx

# --- 設定 ---
# 1. 保存コードで指定したフォルダ名
folder_name = "adj_json_communities_txt" 

# 2. ★分析したいファイル名を指定★
file_to_analyze = "community_0.txt"
# ----------------

# ファイルのフルパスを作成
file_path = os.path.join(folder_name, file_to_analyze)

try:
    # --- 1. JSON形式のTXTファイルを読み込む ---
    print(f"ファイル '{file_path}' を読み込んでいます...")
    with open(file_path, 'r', encoding='utf-8') as f:
        # ご提示のコード(json.dump)で保存したため、json.loadで読み込みます
        adj_dict = json.load(f)

    # --- 2. グラフオブジェクトを復元 ---
    print("nx.from_dict_of_dicts でグラフを復元しています...")
    # この adj_dict は「辞書の辞書」形式なので、この関数で正しく読み込めます
    G = nx.from_dict_of_dicts(adj_dict, create_using=nx.DiGraph)

    # ノード数が3未満かチェック
    if G.number_of_nodes() < 3:
        print(f"ノード数: {G.number_of_nodes()}")
        print("エラー: ノード数が3未満のため、クラスター係数を計算できません。")
    else:
        # --- 3. 平均クラスター係数を計算 ---
        print("平均クラスター係数を計算中...")
        avg_cluster = nx.average_clustering(G)
        
        # --- 4. 結果を表示 ---
        print("\n--- 計算結果 ---")
        print(f"ファイル名: {file_to_analyze}")
        print(f"ノード数: {G.number_of_nodes()}")
        print(f"エッジ数: {G.number_of_edges()}")
        print(f"平均クラスター係数: {avg_cluster:.6f}")

except FileNotFoundError:
    print(f"エラー: ファイル '{file_path}' が見つかりません。")
except json.JSONDecodeError:
    print(f"エラー: ファイル '{file_path}' は正しいJSON形式ではありません。")
except Exception as e:
    # 'int' object is not iterable などのエラーはここで捕捉されます
    print(f"エラー: 処理中に予期せぬ問題が発生しました。 {e}")

ファイル 'adj_json_communities_txt/community_0.txt' を読み込んでいます...
nx.from_dict_of_dicts でグラフを復元しています...
エラー: 処理中に予期せぬ問題が発生しました。 'int' object is not iterable


In [ ]:
import os
import json
import networkx as nx
import numpy as np

# TXTファイルが保存されているフォルダ名
folder_name = "directed_communities_txt" 

# フォルダの存在確認
if not os.path.exists(folder_name):
    print(f"エラー: フォルダ '{folder_name}' が見つかりません。")
    print("先にファイル保存コードを実行してください。")
    exit()

print(f"フォルダ '{folder_name}' 内のファイルを分析します...\n")

try:
    # 拡張子が.txtのファイルを探します
    txt_files = [f for f in os.listdir(folder_name) if f.endswith('.txt')]
    if not txt_files:
        print("エラー: フォルダ内にTXTファイルが見つかりません。")
        exit()
except Exception as e:
    print(f"エラー: フォルダの読み込み中に問題が発生しました。 {e}")
    exit()

# 各コミュニティの統計情報を格納するリスト
community_stats = []

# --- 1. 全てのファイルを読み込み、平均クラスター係数を計算 ---
for filename in txt_files:
    file_path = os.path.join(folder_name, filename)
    
    try:
        # JSON形式の.txtファイルを読み込みます
        with open(file_path, 'r', encoding='utf-8') as f:
            adj_dict = json.load(f)

        # ノード数が3未満（三角形が作れない）場合はスキップ
        if len(adj_dict) < 3:
            continue

        # 辞書から有向グラフを復元
        # クラスター係数の計算では有向グラフ(DiGraph)として扱うのが適切です
        G = nx.from_dict_of_dicts(adj_dict, create_using=nx.DiGraph)

        # 平均クラスター係数を計算
        avg_cluster = nx.average_clustering(G)
        
        # 結果をリストに追加
        community_stats.append({
            "filename": filename,
            "nodes": G.number_of_nodes(),
            "avg_cluster": avg_cluster,
        })

    except Exception as e:
        print(f"エラー: ファイル '{filename}' の処理中に問題が発生しました。 {e}")
        continue # 問題のファイルはスキップ

# --- 2. リストが空でないか確認 ---
if not community_stats:
    print("\n--- 分析結果 ---")
    print("分析対象となるコミュニティが見つかりませんでした。")
    print("（ノード数が3以上のコミュニティが0個でした）")
    exit()

# --- 3. 平均クラスター係数でソート ---
sorted_communities = sorted(community_stats, key=lambda x: x["avg_cluster"])

# --- 4. 3つのコミュニティを特定 ---
# 最も低いコミュニティ
lowest_comm = sorted_communities[0]

# 最も高いコミュニティ
highest_comm = sorted_communities[-1]

# 中央値に最も近いコミュニティ
all_avg_clusters = [c["avg_cluster"] for c in sorted_communities]
median_value = np.median(all_avg_clusters)
median_comm = min(sorted_communities, key=lambda x: abs(x["avg_cluster"] - median_value))

# --- 5. 結果の詳細表示 ---
print(f"--- 分析結果 ({len(community_stats)}個のコミュニティ中) ---")

def print_community_details(title, community_info):
    """コミュニティの詳細情報を表示する関数"""
    print(f"\n◆ {title}")
    print(f"  ファイル名: {community_info['filename']}")
    print(f"  ノード数  : {community_info['nodes']}")
    print(f"  平均クラスター係数: {community_info['avg_cluster']:.6f}")

print_community_details("平均クラスター係数が【最も低い】コミュニティ", lowest_comm)
print_community_details("平均クラスター係数が【中央値に最も近い】コミュニティ", median_comm)
print_community_details("平均クラスター係数が【最も高い】コミュニティ", highest_comm)

フォルダ 'directed_communities_txt' 内のファイルを分析します...

エラー: ファイル 'community_1.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_0.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_2.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_3.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_7.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_6.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_4.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_5.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_13.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_12.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_10.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_38.txt' の処理中に問題が発生しました。 'int' object is not iterable
エラー: ファイル 'community_39.txt' の処理中に問題が発生しました。 'int' object is not iterab

IndexError: list index out of range

: 